Lesson 9

---


# 🧑‍🍳 Interact with a Model

<h3>Attention:</h3>

Only execute the cells of "Step 1: 👷‍♀️ Model building" if you intend to regenerate the model. A pre-existing model is stored in the "models" folder. For utilizing the model, refer to the second section in this notebook, titled "Step 2: 🦾 Model usage" below.

## Step 1: 👷‍♀️ Model Building

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np

iris = datasets.load_iris() #Loading the dataset

print(iris.keys())


What is the Iris dataset?

The iris data consisted of 150 samples of three species of Iris. The first column represented sepal length, the second column represented sepal width, the third column represented petal length, and the fourth column represented petal width.

In [ ]:
type(iris)

### Converting the dataset to pandas dataframe

In [ ]:
df_iris = pd.DataFrame(
    data= np.c_[iris['data'], iris['target']],
    columns= iris['feature_names'] + ['target']
    )

df_iris.head()

 Three of these iris species look similar, but the difference in measurements can be used to classify them. This data set is a classic example of supervised learning. The input variables are sepal length and width and petal length and width; each row represents an instance or observation. The output variable is Iris-setosa, Iris-versicolor, or Iris-virginica; each column represents a class label.

### Species Name

In [ ]:
species = []

for i in range(len(iris['target'])):
    if iris['target'][i] == 0:
        species.append("setosa")
    elif iris['target'][i] == 1:
        species.append('versicolor')
    else:
        species.append('virginica')


df_iris['species'] = species

df_iris.head()

### Exploratory Data Analysis

In [ ]:
df_iris.groupby('species').size()

> Each number of classes has 50 instances together constituting 150 in total.

In [ ]:
df_iris.describe()

In [ ]:
import matplotlib.pyplot as plt

setosa = df_iris[df_iris["species"] == "setosa"]
versicolor = df_iris[df_iris["species"] =='versicolor']
virginica = df_iris[df_iris["species"] =='virginica']

fig, ax = plt.subplots()
fig.set_size_inches(13, 7) # adjusting the length and width of plot

# labels and scatter points
ax.scatter(setosa['petal length (cm)'], setosa['petal width (cm)'], label="Setosa", facecolor="blue")
ax.scatter(versicolor['petal length (cm)'], versicolor['petal width (cm)'], label="Versicolor", facecolor="green")
ax.scatter(virginica['petal length (cm)'], virginica['petal width (cm)'], label="Virginica", facecolor="red")


ax.set_xlabel("petal length (cm)")
ax.set_ylabel("petal width (cm)")
ax.grid()
ax.set_title("Iris petals")
ax.legend()

### Create a classification model

In [ ]:
from sklearn.model_selection import train_test_split

# Dropping the target and species since we only need the measurements
X = df_iris.drop(['target','species'], axis=1)

# X features are: (1) petal length (cm) and (2) petal width (cm)

# converting into numpy array and assigning petal length and petal width
X = X.to_numpy()[:, (2,3)]
y = df_iris['target']

# Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5, random_state=42)

In [ ]:
X

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg_model = LogisticRegression()
log_reg_model.fit(X_train,y_train)

print("Model is trained")

### Use the model to predict Training and Test sets

In [ ]:
training_prediction = log_reg_model.predict(X_train)
training_prediction

In [ ]:
test_prediction = log_reg_model.predict(X_test)
test_prediction

### Performance Measures

In [ ]:
from sklearn import metrics

print("Precision, Recall, Confusion matrix, in training\n")

# Precision Recall scores
print(metrics.classification_report(y_train, training_prediction, digits=3))

# Confusion matrix
print(metrics.confusion_matrix(y_train, training_prediction))

In [ ]:
print("Precision, Recall, Confusion matrix, in testing\n")

# Precision Recall scores
print(metrics.classification_report(y_test, test_prediction, digits=3))

# Confusion matrix
print(metrics.confusion_matrix(y_test, test_prediction))


### Save the model

In [ ]:
import pickle
path = "./models/iris_model.pkl"

with open(path, 'wb') as file:
    pickle.dump(log_reg_model, file)

print(f"✅ File {path} was saved successfully")

### Save a model wrapper class

In [ ]:
features = ["petal length (cm)", "petal width (cm)"]
species_label = ['setosa', 'versicolor', 'virginica']

class ClassifierWrapper:
    def __init__(self, model, features, class_labels):
        self.model = model
        self.features = features
        self.class_labels = class_labels

    def predict(self, x_observation: list) -> str:
        result = self.model.predict([x_observation])
        return self.class_labels[int(result[0])]

    def prediction_needs(self, verbosity=True):
        if verbosity : return f"You need to provide the values of {self.features} to get a prediction."
        else : return self.features


iris_classifier = ClassifierWrapper(
    model=log_reg_model,
    features=features,
    class_labels=species_label
)


iris_classifier.predict([1.2, 0.2])


In [ ]:
path = "./models/iris_classifier.pkl"

with open(path, 'wb') as file:
    pickle.dump(iris_classifier, file)

print(f"✅ File {path} was saved successfully")

## Step 2: 🦾 Model usage

### Load the model

In [1]:
import pickle

path = "./models/iris_model.pkl"

with open(path, 'rb') as file:
    iris_model_loaded = pickle.load(file)


iris_model_loaded.__repr__()

'LogisticRegression()'

In [2]:
# X features are: (1) petal length (cm) and (2) petal width (cm)
features = ["petal length (cm)", "petal width (cm)"]

result = iris_model_loaded.predict([[1.2, 0.2]])

species_label = ['setosa', 'versicolor', 'virginica']
index = int(result[0])

print(f"index: {index}")
species_label[index]

index: 0


'setosa'

### Load the Model Wrapper Class

In [6]:
import pickle
from classifier_wrapper import ClassifierWrapper

path = "./models/iris_classifier.pkl"

with open(path, 'rb') as f:
    iris_classifier_loaded = pickle.load(f)


iris_classifier_loaded.__repr__()

'<classifier_wrapper.ClassifierWrapper object at 0x146962140>'

In [36]:
iris_classifier_loaded.prediction_needs()

"You need to provide the values of ['petal length (cm)', 'petal width (cm)'] to get a prediction."

In [ ]:
iris_classifier_loaded.predict([1.2, 0.2])

## Step 3: 🤖+🧠 Chat and Classifier

In [8]:
from util import local_settings
from openai import OpenAI

print("First LLM API example")
print(f"✅ OpenAI Key loaded ({local_settings.OPENAI_API_KEY[0:-15]}...)")

client = OpenAI(api_key=local_settings.OPENAI_API_KEY)

First LLM API example
✅ OpenAI Key loaded (sk-gR3MSxngK0Vfbay3JQopT3BlbkFJ1S9XN...)


In [9]:
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate

prompt_template = ChatPromptTemplate.from_template("")
print(prompt_template)

input_variables=[] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=''))]


In [10]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0, messages=None):
    if not messages:
        messages = [{"role": "user", "content": prompt}]
    else:
        messages.append({"role": "user", "content": prompt})

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

    return completion.choices[0].message.content

In [28]:
messages = [{"role" : "system",
            "content" :"""
You are a friendly assistant and your objective is keep the answer as simple as you can.

If is asked to you if you can classify Iris flowers, you must say yes and follow the instructions of Task 1 below indicated inside a triple backticks.

```Task 1

Title: Predict a Iris flower by species

Instructions:
- you need to request two information 'petal length (cm)', 'petal width (cm)' which are the input of the classification model.
- You also must add an final annotation <<<IRIS_CLASSIFICATION_ON>>>
- Get the values provided and generate a list of values in the order they were provided. e.g., [0.2, 0.3]

Example of request message:
<message>
Yes, I can classify an Iris flower. To do that, I need two pieces of information from you, please inform:
a) petal length (cm)
b) petal width (cm).

<<<IRIS_CLASSIFICATION_ON>>>
</message>

```
"""}]


In [29]:
prompt = "Hi, I am Fernando"
response = get_completion(prompt, temperature=1, messages=messages)
print(response)
messages.append({"role":"assistant","content": response})

Hello Fernando! How can I assist you today?


In [30]:
prompt = "Could you classify a Iris flower?"
response = get_completion(prompt, temperature=1, messages=messages)
print(response)
messages.append({"role":"assistant","content": response})

Yes, I can classify an Iris flower. To do that, I need two pieces of information from you, please inform:
a) petal length (cm)
b) petal width (cm).

<<<IRIS_CLASSIFICATION_ON>>>


In [31]:
IRIS_CLASSIFICATION_ON = False
# if the command is identified
if response.find("<<<IRIS_CLASSIFICATION_ON>>>") >= 0 :
    print("IRIS_CLASSIFICATION_ON")
    IRIS_CLASSIFICATION_ON = True

IRIS_CLASSIFICATION_ON


In [32]:
prompt = "a) 1.2, b). 0.2"
response = get_completion(prompt, temperature=1, messages=messages)
print(response)
messages.append(response)

Thank you for providing the information. Based on the petal length of 1.2 cm and petal width of 0.2 cm, the classification of the Iris flower is: Versicolor.


In [33]:

template = f"""
Extract the petal length (cm) and petal width (cm) from the following text and generate a list with these values.

TEXT: {response}

EXAMPLE:
- petal length (cm): 1.2
- petal width (cm): 0.2

OUTPUT: a list
[1.2, 0.2]
"""

print(template)


if IRIS_CLASSIFICATION_ON:
    r2 = get_completion(template, temperature=1,)



Extract the petal length (cm) and petal width (cm) from the following text and generate a list with these values.

TEXT: Thank you for providing the information. Based on the petal length of 1.2 cm and petal width of 0.2 cm, the classification of the Iris flower is: Versicolor.

EXAMPLE:
- petal length (cm): 1.2
- petal width (cm): 0.2

OUTPUT: a list
[1.2, 0.2]



In [34]:
if IRIS_CLASSIFICATION_ON:
    print(r2)
    print(type(r2))
    print(type(eval(r2)))

    observation_to_predict = eval(r2)
    print(observation_to_predict)

[1.2, 0.2]
<class 'str'>
<class 'list'>
[1.2, 0.2]


In [35]:
prediction = ""

prompt_template_3 = """
based on the input
"""

if IRIS_CLASSIFICATION_ON:

    prediction = iris_classifier_loaded.predict(observation_to_predict)

    IRIS_CLASSIFICATION_ON = False

print(prediction)

setosa
